In [1]:
from kiteconnect import KiteTicker, KiteConnect
import datetime
import sys
import pandas as pd
import os
import sqlite3
from auto_login import get_curr_path,get_credentials,auto_login,generate_access_token

In [2]:
folder_name = 'api_keys'
curr_dir = get_curr_path(folder_name)

file_name = 'credentials.txt'
api_key,api_secret,user_name,pwd,totp_key = get_credentials(curr_dir,file_name)

request_token = auto_login(api_key,user_name,pwd,totp_key)

access_token = generate_access_token(request_token,api_key,api_secret)

In [3]:
kite = KiteConnect(api_key=api_key)
kite.set_access_token(access_token=access_token)
instrument_dump = kite.instruments('NSE')
instrument_df = pd.DataFrame(instrument_dump)
instrument_df.to_csv('NSE_instruments.csv',index=False)

In [4]:
db = sqlite3.connect('ticks.db')

In [5]:
db

In [6]:
def create_tables(tokens):
    c = db.cursor() # cursor executes queries 
    for i in tokens:
        c.execute("CREATE TABLE IF NOT EXISTS TOKEN{} (ts datetime primary key, price real(15,5), volume integer)".format(i))
    try:
        db.commit()
    except:
        db.rollback() 

In [7]:
def insert_ticks(ticks):
    c = db.cursor()
    for tick in ticks:
        try:
            tok = "TOKEN"+ str(tick['instrument_token'])
            vals = [tick['timestamp'],tick['last_price'],tick['volume']]
            query = "INSERT INTO {} (ts,price,volume) VALUES (?,?,?)".format(tok)
            c.execute(query,vals)
        except:
            pass 
        try: 
            db.commit()
        except:
            db.rollback()

In [8]:
instrument_dump = kite.instruments("NSE")
instrument_df = pd.DataFrame(instrument_dump)

In [9]:
instrument_df.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE
1,256777,1003,NIFTY MIDCAP 100,NIFTY MIDCAP 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
2,260105,1016,NIFTY BANK,NIFTY BANK,0.0,,0.0,0.0,0,EQ,INDICES,NSE
3,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.0,0,EQ,INDICES,NSE
4,257033,1004,NIFTY DIV OPPS 50,NIFTY DIV OPPS 50,0.0,,0.0,0.0,0,EQ,INDICES,NSE


In [10]:
def tokenLookup(instrument_df,tickers):
    for symbol in tickers:
        filtered_df = instrument_df[instrument_df['tradingsymbol'] == symbol]
    token_list = []
    token_list.append(int(filtered_df['instrument_token'].values[0]))
    return token_list


In [11]:
def tokenLookup(instrument_df, symbol_list):
    token_list = []
    for symbol in symbol_list:
        # Filter the DataFrame for the matching tradingsymbol
        filtered_df = instrument_df[instrument_df['tradingsymbol'] == symbol]
        
        # Check if the filtered DataFrame is empty
        if filtered_df.empty:
            print(f"Warning: No matching token found for symbol {symbol}.")
            continue
        
        # Append the first matching token to the list
        token_list.append(int(filtered_df.instrument_token.values[0]))
    
    return token_list


In [12]:
tickers = ["TCS","INFY","ICICIBANK","TATAMOTORS","TCS","HDFCBANK"]

In [13]:
kws = KiteTicker(api_key=api_key,access_token=access_token)
kws

In [14]:
tokens = tokenLookup(instrument_df,tickers)

In [15]:
tokens

[2953217, 408065, 1270529, 884737, 2953217, 341249]

In [16]:
create_tables(tokens)

In [17]:
c = db.cursor()

In [18]:
c.execute("SELECT name FROM sqlite_master WHERE type ='table'")
tables = c.fetchall()
for table in tables:
    print(table[0])

TOKEN2953217
TOKEN408065
TOKEN1270529
TOKEN884737
TOKEN341249


In [19]:
def on_ticks(ws,ticks):
    insert_tick = insert_ticks(ticks) 
    print(ticks)

def on_connect(ws,response):
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL,tokens)

while True:
    now = datetime.datetime.now()
    if (now.hour >= 9 and now.minute >= 15):
        kws.on_ticks = on_ticks
        kws.on_connect = on_connect
        kws.connect()
    if (now.hour >= 15 and now.minute >= 30):
        sys.exit()
    


[{'tradable': True, 'mode': 'full', 'instrument_token': 341249, 'last_price': 1593.65, 'last_traded_quantity': 115, 'average_traded_price': 1586.77, 'volume_traded': 4863509, 'total_buy_quantity': 435788, 'total_sell_quantity': 1429117, 'ohlc': {'open': 1584.0, 'high': 1597.0, 'low': 1577.7, 'close': 1580.75}, 'change': 0.8160683219990569, 'last_trade_time': datetime.datetime(2024, 6, 14, 13, 19, 54), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'exchange_timestamp': datetime.datetime(2024, 6, 14, 13, 19, 54), 'depth': {'buy': [{'quantity': 64, 'price': 1593.65, 'orders': 1}, {'quantity': 29, 'price': 1593.6, 'orders': 1}, {'quantity': 33, 'price': 1593.55, 'orders': 2}, {'quantity': 285, 'price': 1593.5, 'orders': 3}, {'quantity': 20, 'price': 1593.45, 'orders': 1}], 'sell': [{'quantity': 173, 'price': 1593.7, 'orders': 5}, {'quantity': 60, 'price': 1593.75, 'orders': 1}, {'quantity': 11, 'price': 1593.8, 'orders': 2}, {'quantity': 134, 'price': 1594.0, 'orders': 2}, {'quantity': 125, 

Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


ReactorNotRestartable: 

In [ ]:
c.execute('SELECT name from sqlite_master where type= "table"')
c.fetchall()

c.execute('''PRAGMA table_info(TOKEN975873)''')
c.fetchall()

for m in c.execute('''SELECT * FROM TOKEN975873'''):
    print(m)

In [ ]:
db.close()